# 03. RAG Generation (Orchestration Only)

This notebook orchestrates:

- Retrieval (vector / LLM rerank)
- Context construction
- LLM generation with citation validation + optional two-pass retry
- Optional run logging

All core logic lives in `src/`. This notebook only wires components together.


In [14]:
from pathlib import Path
import sys

# ---- project root (invest-rag/) ----
NOTEBOOK_DIR = Path.cwd()
PROJECT_DIR = NOTEBOOK_DIR.parent  # notebooks/ 상위 폴더

assert (PROJECT_DIR / "src").exists(), f"src folder not found under PROJECT_DIR={PROJECT_DIR}"

# IMPORTANT: enable `from src...` imports
if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))

# ---- artifacts ----
INDEX_DIR = PROJECT_DIR / "indexes" / "faiss"
INDEX_PATH = INDEX_DIR / "index.bin"
META_PATH  = INDEX_DIR / "meta.jsonl"

for p in [INDEX_PATH, META_PATH]:
    assert p.exists(), f"Missing artifact: {p}"

# ---- retrieval modules ----
from src.llm.embedding import embed_query
from src.retrieval.vector_store import VectorStore
from src.eval.search_wrappers import (
    make_vectorstore_search_fn,
    make_llm_rerank_search_fn,
)

vs = VectorStore.load(index_path=INDEX_PATH, meta_path=META_PATH)

vector_search_fn = make_vectorstore_search_fn(vs, embed_query=embed_query, normalize=True)
rerank_search_fn = make_llm_rerank_search_fn(vector_search_fn, k_vec=10)

print("PROJECT_DIR:", PROJECT_DIR)
print("Artifacts:", INDEX_DIR)
print("Ready: vector_search_fn / rerank_search_fn")


PROJECT_DIR: c:\Users\CG\Desktop\invest-rag
Artifacts: c:\Users\CG\Desktop\invest-rag\indexes\faiss
Ready: vector_search_fn / rerank_search_fn


## Context + Generation

We reuse the existing modules:

- `src/llm/context.py`: `build_context(results) -> str`
- `src/llm/generate.py`: `rag_generate_with_retry(query, context, retrieved) -> (answer, ok)`


In [15]:
from src.llm.context import build_context
from src.llm.generate import rag_generate_with_retry

## Run Logging (Optional)

Logs each run as a JSONL row under `logs/run_logs.jsonl`.


In [16]:
import datetime
import json

RUNLOG_PATH = PROJECT_DIR / "logs" / "run_logs.jsonl"

def log_run(payload: dict, path: Path = RUNLOG_PATH) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    row = {"ts": datetime.datetime.now().isoformat(timespec="seconds"), **payload}
    with path.open("a", encoding="utf-8") as f:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")


## Orchestration

- Choose retrieval mode: vector vs rerank
- Retrieve top-k results
- Build context (from `src/llm/context.py`)
- Generate grounded answer with citations (from `src/llm/generate.py`)
- Return answer + sources list


In [17]:
USE_RERANK = False
K = 5

def retrieve(query: str, k: int = K, use_rerank: bool = USE_RERANK):
    fn = rerank_search_fn if use_rerank else vector_search_fn
    return fn(query, k)

def format_sources(results: list[dict], max_items: int = 8) -> list[str]:
    """Minimal helper (not retrieval logic): pretty-print sources."""
    seen = set()
    out = []
    for r in results:
        doc_id = r.get("doc_id")
        if not doc_id:
            continue
        title = (r.get("title") or "").strip()
        date  = (r.get("date") or "").strip()
        src   = (r.get("source") or "").strip()

        s = f"[{doc_id}] {title}".strip()
        tail = ", ".join([x for x in [date, src] if x])
        if tail:
            s += f" ({tail})"

        if s not in seen:
            seen.add(s)
            out.append(s)
        if len(out) >= max_items:
            break
    return out

def run_query(query: str, *, use_rerank: bool = USE_RERANK, k: int = K) -> dict:
    retrieved = retrieve(query, k=k, use_rerank=use_rerank)

    # ✅ context module (no duplicated logic)
    context = build_context(retrieved)

    if not context.strip():
        return {
            "query": query,
            "answer": "Not enough context. Retrieved empty context.",
            "ok": False,
            "sources": [],
            "retrieved": retrieved,
        }

    # ✅ generation module (includes citation validation + retry)
    answer, ok = rag_generate_with_retry(query=query, context=context, retrieved=retrieved)

    # optional logging
    log_run({
        "query": query,
        "k_ctx": k,
        "use_rerank": use_rerank,
        "ok": ok,
        "retrieved": [
            {
                "rank": r.get("rank"),
                "doc_id": r.get("doc_id"),
                "chunk_id": r.get("chunk_id"),
                "score": r.get("score"),
                "title": r.get("title"),
            }
            for r in retrieved
        ],
        "answer": answer,
    })

    return {
        "query": query,
        "answer": answer,
        "ok": ok,
        "sources": format_sources(retrieved),
        "retrieved": retrieved,
    }


## Grounding Strategy (Two-Pass)

`rag_generate_with_retry()` implements a two-pass strategy:

1. First pass: generate answer with `[doc_id]` citations.
2. Validate citations against retrieved doc_ids.
3. If invalid → retry while restricting allowed doc_ids.

This reduces hallucinated citations while keeping latency low.


## Demo


In [18]:
q = "HBM 공급 타이트가 메모리 업체 실적에 미치는 영향은?"

out = run_query(q, use_rerank=False, k=5)

print(out["answer"])
print("\nValid citations:", out["ok"])
print("\nSources:")
print("\n".join(out["sources"]))


HBM 공급 타이트는 메모리 업체 실적에 다음과 같은 영향을 미칩니다:

1. 출하량 증가에 제한: NSIL의 3분기 출하는 전분기 대비 증가가 예상되나, HBM 수급 제약으로 상단이 제한적입니다. 즉, 공급 부족이 출하량 확대를 제약하는 요인으로 작용합니다[report_0001].

2. 출하 믹스 개선 가능성: HBM 공급 타이트가 지속되면서 고대역폭 메모리 탑재 비중이 높은 AI 가속기 비중 확대가 추진되고, 이로 인해 출하 믹스가 상향될 가능성이 있습니다. 믹스 개선은 평균판매가격(ASP) 개선으로 이어질 수 있습니다[news_0001][report_0001].

3. 비용 및 납기 리스크 증가: 공급 불확실성으로 인해 긴급 조달 및 물류 비용 부담이 증가하고, 납기 리스크가 상존합니다[news_0001][report_0001].

4. CAPEX 상향 및 단기 부담: HBM 증설 경쟁 심화로 메모리 업체들은 CAPEX를 상향할 가능성이 있으며, 단기적으로 감가상각 부담과 현금흐름 부담이 확대될 수 있습니다. 이는 단기 실적에 부담으로 작용할 수 있으나, 장기적으로는 HBM 시장 내 포지셔닝 강화에 기여할 전망입니다[news_0005][report_0006].

종합하면, HBM 공급 타이트는 출하량 증가를 제한하는 반면, 고부가가치 제품 중심의 출하 믹스 개선과 ASP 상승 가능성을 제공하지만, 비용 부담과 납기 리스크가 증가하여 단기 실적에 복합적인 영향을 미칩니다. 또한, 증설 경쟁으로 인한 CAPEX 증가가 단기 부담 요인으로 작용할 수 있습니다.

Valid citations: True

Sources:
[news_0005] HBM 증설 경쟁 심화, CAPEX 상향 가능성 (2025-09-10, news_summary)
[news_0001] HBM 공급 타이트 지속, 3Q 출하 믹스 상향 가능성 (2025-08-03, news_summary)
[report_0001] 3Q 가이던스: 출하 증가 vs HBM 제약 (2025-08-07, repor

### Batch demo (optional)


In [19]:
demo_queries = [
    "출하량보다 제품 믹스가 더 중요하다고 평가된 사례는?",
    "데이터센터 전력/냉각 제약이 단기 병목이라는 리스크는?",
    "HBM 증설 경쟁 심화로 CAPEX 부담이 커질 수 있다는 내용은?",
]

for q in demo_queries:
    out = run_query(q, use_rerank=False, k=5)
    print("\n" + "="*80)
    print("Q:", q)
    print("OK:", out["ok"])
    print("A:", out["answer"])
    print("Sources:", " | ".join(out["sources"]))



Q: 출하량보다 제품 믹스가 더 중요하다고 평가된 사례는?
OK: True
A: 출하량보다 제품 믹스가 더 중요하다고 평가된 사례는 2025년 8월 7일 NSIL 보고서에서 확인할 수 있습니다. 해당 보고서에서는 3분기 출하가 QoQ 증가가 예상되나 HBM 수급 제약으로 상단은 제한적이며, 마진은 고급 SKU 비중 확대로 개선 가능하나 공급 불확실성으로 비용 리스크가 존재한다고 언급했습니다. 이 보고서에서 투자 포인트는 ‘출하량’보다 ‘믹스’에 있다고 판단한다고 명확히 평가했습니다[report_0001].
Sources: [news_0008] 고객사 재고 조정 마무리, 4Q 계약가 협상 주목 (2025-10-05, news_summary) | [report_0008] 4Q 전망: 가동률 회복과 제품 믹스 정상화 (2025-10-06, report_excerpt) | [report_0001] 3Q 가이던스: 출하 증가 vs HBM 제약 (2025-08-07, report_excerpt) | [news_0003] 선단 공정 수율 개선, 2H 가동률 회복 기대 (2025-08-19, news_summary) | [news_0001] HBM 공급 타이트 지속, 3Q 출하 믹스 상향 가능성 (2025-08-03, news_summary)

Q: 데이터센터 전력/냉각 제약이 단기 병목이라는 리스크는?
OK: True
A: 데이터센터 전력 및 냉각 인프라 제약은 AI 서버 수요가 견조함에도 불구하고 단기 출하에 병목으로 작용할 수 있는 주요 리스크로 지적되고 있습니다. NSIL의 출하 모멘텀은 유효하나, 고객사의 증설 속도에 따라 분기별 변동성이 확대될 수 있으며, 전력/냉각 병목이 단기 출하를 제약할 가능성이 있습니다. 또한 경쟁사의 신제품 출시가 집중되는 시기에는 SW 생태계 및 고객 락인 전략이 점유율 방어의 핵심이 될 것으로 보입니다[news_0004][report_0004].
Sources: [news_0004] AI 서버 수요 견조, 다만 전력/냉각 제약이 변

## Vector vs Rerank comparison

This section runs the same query twice:

- **Vector baseline** (`use_rerank=False`)
- **LLM rerank** (`use_rerank=True`)

It then compares:
- retrieved top-k (rank / score / doc_id / title)
- answer + citation validity


In [20]:
# --- helpers for demo output (orchestration-only) ---
def preview_retrieved(retrieved, max_rows=5):
    rows = []
    for r in (retrieved or [])[:max_rows]:
        rows.append({
            "rank": r.get("rank"),
            "score": r.get("score"),
            "doc_id": r.get("doc_id"),
            "title": (r.get("title") or "")[:90],
        })
    return rows

def compare_top1(vec_retrieved, rr_retrieved):
    v1 = (vec_retrieved or [{}])[0].get("doc_id")
    r1 = (rr_retrieved  or [{}])[0].get("doc_id")
    return {"vector_top1": v1, "rerank_top1": r1, "changed": (v1 != r1)}


In [21]:
# --- run comparison ---
QUERY = "HBM 공급 타이트가 메모리 업체 실적에 미치는 영향은?"

out_vec = run_query(QUERY, use_rerank=False, k=5)
out_rr  = run_query(QUERY, use_rerank=True,  k=5)

print("QUERY:", QUERY)
print("\n=== Retrieval top-5 (vector) ===")
print(preview_retrieved(out_vec.get("retrieved", []), max_rows=5))

print("\n=== Retrieval top-5 (rerank) ===")
print(preview_retrieved(out_rr.get("retrieved", []), max_rows=5))

print("\n=== Top-1 change ===")
print(compare_top1(out_vec.get("retrieved", []), out_rr.get("retrieved", [])))

print("\n=== Answer (vector) ===")
print(out_vec.get("answer",""))
print("Citations OK:", out_vec.get("ok"))
print("Sources:")
print("\n".join(out_vec.get("sources", [])))

print("\n=== Answer (rerank) ===")
print(out_rr.get("answer",""))
print("Citations OK:", out_rr.get("ok"))
print("Sources:")
print("\n".join(out_rr.get("sources", [])))


QUERY: HBM 공급 타이트가 메모리 업체 실적에 미치는 영향은?

=== Retrieval top-5 (vector) ===
[{'rank': 1, 'score': 0.6080902814865112, 'doc_id': 'news_0005', 'title': 'HBM 증설 경쟁 심화, CAPEX 상향 가능성'}, {'rank': 2, 'score': 0.5481195449829102, 'doc_id': 'news_0001', 'title': 'HBM 공급 타이트 지속, 3Q 출하 믹스 상향 가능성'}, {'rank': 3, 'score': 0.47888419032096863, 'doc_id': 'report_0001', 'title': '3Q 가이던스: 출하 증가 vs HBM 제약'}, {'rank': 4, 'score': 0.4731154143810272, 'doc_id': 'report_0006', 'title': 'CAPEX 상향: 단기 부담 vs 장기 HBM 포지셔닝'}, {'rank': 5, 'score': 0.4496529698371887, 'doc_id': 'disc_0002', 'title': 'HBM 생산라인 증설 계획(요약)'}]

=== Retrieval top-5 (rerank) ===
[{'rank': 1, 'score': 0.5481195449829102, 'doc_id': 'news_0001', 'title': 'HBM 공급 타이트 지속, 3Q 출하 믹스 상향 가능성'}, {'rank': 2, 'score': 0.6080902814865112, 'doc_id': 'news_0005', 'title': 'HBM 증설 경쟁 심화, CAPEX 상향 가능성'}, {'rank': 3, 'score': 0.47888419032096863, 'doc_id': 'report_0001', 'title': '3Q 가이던스: 출하 증가 vs HBM 제약'}, {'rank': 4, 'score': 0.4731154143810272, 'doc_id': 

## Retrieval Comparison (Vector vs Rerank)

For the same query, both vector search and LLM rerank retrieve largely overlapping top-k candidates.

However:

- **Vector top-1:** news_0005 (CAPEX / 증설 경쟁 관점)
- **Rerank top-1:** news_0001 (공급 타이트 / 출하 믹스 관점)

This indicates that rerank promotes documents that are more directly aligned with the question intent, 
while preserving the same candidate pool.

Since generation uses top-k context, final answers remain stable and grounded 
in multiple supporting documents.

This behavior is desirable:
- Retrieval remains robust.
- Rerank refines prioritization without destabilizing outputs.